<a href="https://colab.research.google.com/github/jnardosp/mc_asymptotic_analysis/blob/main/funcion_recursiva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Código Python para calcular la distribución estacionaria usando el método de tiempos medios de regreso.
# Ejecuta esto tal cual; está escrito para ser legible y con comprobaciones básicas.
import numpy as np

def stationary_via_return_times(P, tol=1e-12, use_lstsq_on_singular=True):
    r"""
    Calcula la distribución estacionaria de una matriz de transición P
    usando el método de tiempos medios de regreso.

    Parámetros
    ----------
    P : array-like, shape (n, n)
        Matriz de transición (filas suman 1).
    tol : float
        Tolerancia para comprobaciones numéricas.
    use_lstsq_on_singular : bool
        Si True, en caso de que la matriz del sistema sea singular se resuelve
        con lstsq (solución de mínimos cuadrados). Si False, se lanza excepción.

    Retorna
    -------
    pi : ndarray, shape (n,)
        Vector de probabilidad estacionaria (suma 1).
    r : ndarray, shape (n,)
        Vector de tiempos medios de regreso r_i = E_i[T_i^+].

    Notas
    -----
    - Si la cadena es reducible o algunos tiempos medios son infinitos, el
      método puede fallar o producir soluciones numéricas inestables.
    - El sistema resuelto para cada i es (I - P_{S\{i\}, S\{i\}}) t = 1.
    """
    P = np.array(P, dtype=float)
    if P.ndim != 2 or P.shape[0] != P.shape[1]:
        raise ValueError("P debe ser una matriz cuadrada (n x n).")
    n = P.shape[0]
    # Comprobación simple: filas suman 1 (o muy cerca)
    row_sums = P.sum(axis=1)
    if not np.allclose(row_sums, np.ones(n), atol=1e-9):
        print("Advertencia: las filas de P no suman 1 con precisión numérica. Sumas de filas:", row_sums)

    r = np.empty(n, dtype=float)  # tiempos medios de regreso
    for i in range(n):
        # índices de estados distintos de i
        idx = [j for j in range(n) if j != i]
        if len(idx) == 0:
            # caso n=1 (cadena trivial)
            r[i] = 1.0  # T_i^+ = 1 (vuelve en 1 paso siempre)
            continue
        # Matriz y vector del sistema (I - P_sub) t = 1
        A = np.eye(n-1) - P[np.ix_(idx, idx)]
        b = np.ones(n-1)
        try:
            t_sub = np.linalg.solve(A, b)  # t_sub corresponde a t_{j,i} para j in idx
        except np.linalg.LinAlgError:
            if not use_lstsq_on_singular:
                raise
            # Intento de mínimos cuadrados (puede indicar tiempos medios infinitos o cadena no recurrente)
            t_sub, residuals, rank, s = np.linalg.lstsq(A, b, rcond=None)
            print(f"Atención: sistema singular o mal condicionado para i={i}. Se usó lstsq (rank={rank}).")
        # Construimos r_i = 1 + sum_j P_{i j} t_{j,i}, recordando que t_{i,i}=0
        r_i = 1.0 + np.dot(P[i, idx], t_sub)
        r[i] = r_i
    # Convertimos a pi: pi_i proportional a 1/r_i
    with np.errstate(divide='ignore', invalid='ignore'):
        inv_r = 1.0 / r
    if np.any(~np.isfinite(inv_r)):
        raise ValueError("Algunos tiempos medios de regreso son infinitos o no finitos; "
                         "la distribución estacionaria no existe (cadena no recurrente positiva).")
    pi = inv_r / np.sum(inv_r)
    return pi, r

def stationary_via_eig(P):
    """Método alternativo por autovector izquierdo (P^T v = v)."""
    P = np.array(P, dtype=float)
    w, v = np.linalg.eig(P.T)
    # Buscamos autovalor más cercano a 1
    idx = np.argmin(np.abs(w - 1.0))
    vec = np.real(v[:, idx])
    if np.allclose(vec, 0):
        raise ValueError("No se encontró autovector real asociado a autovalor 1.")
    vec = vec / np.sum(vec)
    # A veces el autovector viene negativo en signo; forzamos no-negatividad si corresponde
    if np.any(vec < -1e-12):
        # normalizamos por su suma absoluta si hay negativos pequeños; de lo contrario, avisamos
        print("Aviso: el autovector tiene componentes negativas (posible multiplicidad o inestabilidad).")
    vec = np.real_if_close(vec)
    # Aseguramos no-negatividad simple (siempre que sea razonable)
    vec = np.abs(vec)
    vec = vec / np.sum(vec)
    return vec

# Ejemplo: la matriz 3x3 del ejemplo visto en la explicación anterior
P_example = np.array([
    [0.5, 0.3, 0.2],
    [0.2, 0.5, 0.3],
    [0.1, 0.4, 0.5]
])

pi_return, r = stationary_via_return_times(P_example)
pi_eig = stationary_via_eig(P_example)

print("Matriz P de ejemplo:\n", P_example)
print("\nTiempos medios de regreso r_i = E_i[T_i^+]:", r)
print("\nDistribución estacionaria por método de tiempos de regreso:")
print(pi_return)
print("\nDistribución estacionaria por autovector (verificación):")
print(pi_eig)
print("\nComprobación: pi * P (debe ser igual a pi):")
print(np.round(pi_return @ P_example, 10))
print("\nSuma de pi (debe ser 1):", np.sum(pi_return))

# Si quieres usar la función con otra matriz, reemplaza P_example por tu matriz P.
# Ejemplo de uso:
# P = [[...], [...], ...]
# pi, r = stationary_via_return_times(P)
# print(pi)

Matriz P de ejemplo:
 [[0.5 0.3 0.2]
 [0.2 0.5 0.3]
 [0.1 0.4 0.5]]

Tiempos medios de regreso r_i = E_i[T_i^+]: [4.23076923 2.39130435 2.89473684]

Distribución estacionaria por método de tiempos de regreso:
[0.23636364 0.41818182 0.34545455]

Distribución estacionaria por autovector (verificación):
[0.23636364 0.41818182 0.34545455]

Comprobación: pi * P (debe ser igual a pi):
[0.23636364 0.41818182 0.34545455]

Suma de pi (debe ser 1): 1.0
